### Installing DuckDB and magic_duckdb extension

Download data from here:
https://www.kaggle.com/datasets/catalystcooperative/pudl-project?select=hourly_emissions_epacems.parquet

In [1]:
!python3 -m venv .duckvenv
!source .duckvenv/bin/activate

In [2]:
!pip install duckdb magic_duckdb polars plotly_express nbformat --quiet --user

In [20]:
import duckdb
import pandas as pd
import polars as pl
import plotly_express as px

%load_ext magic_duckdb

The magic_duckdb extension is already loaded. To reload it, use:
  %reload_ext magic_duckdb


Read more about Polars here:
https://pola.rs/#

## Analyzing Data with DuckDB

### SQL queries using DuckDB:
To run SQL queries in DuckDB we can directly use ".sql", no need to create a connection to ":memory:" 

Every DataFrame inside this notebook will be instantly available for DuckDB to make SQL queries against.

### Starting with something simple 

In [4]:
df = pd.DataFrame({
    'column1': [1, 2, 3, 4, 5], 
    'column2': ['a', 'b', 'c', 'd', 'e'], 
})

In [5]:
df.head()

,column1,column2
0,1,a
1,2,b
2,3,c
3,4,d
4,5,e


In [6]:
duckdb.sql('''
SELECT * 
FROM df
''')

┌─────────┬─────────┐
│ column1 │ column2 │
│  int64  │ varchar │
├─────────┼─────────┤
│       1 │ a       │
│       2 │ b       │
│       3 │ c       │
│       4 │ d       │
│       5 │ e       │
└─────────┴─────────┘

- DuckDB was able to query our Pandas DataFrame without any extra steps

- Let's go directly from "FROM..." leaving  "SELECT *" as a legacy

In [7]:
duckdb.sql('''
FROM df 
''')

┌─────────┬─────────┐
│ column1 │ column2 │
│  int64  │ varchar │
├─────────┼─────────┤
│       1 │ a       │
│       2 │ b       │
│       3 │ c       │
│       4 │ d       │
│       5 │ e       │
└─────────┴─────────┘

In [7]:
duckdb.sql('FROM df')

┌─────────┬─────────┐
│ column1 │ column2 │
│  int64  │ varchar │
├─────────┼─────────┤
│       1 │ a       │
│       2 │ b       │
│       3 │ c       │
│       4 │ d       │
│       5 │ e       │
└─────────┴─────────┘

We installed duckdb magic earlier and can now avoid repeating `duckdb.sql` and use:
- `%dql` for single line queries 
&
- `%%dql` for multiple lined queries instead

In [8]:
%%dql
FROM df
SELECT *

,column1,column2
0,1,a
1,2,b
2,3,c
3,4,d
4,5,e


In [9]:
%dql FROM df

,column1,column2
0,1,a
1,2,b
2,3,c
3,4,d
4,5,e


Note that by using the magic_duckdb extension, our queries return a Pandas DataFrame, meaning we are still making use of the "lazyness" of DuckDB during the query, but then immediately bringing the query result entirely to memory.

To avoid this, we can set the type of return by using "-t" followed by the type, choosing from "df", "arrow", "pl", "describe", "show" and "relation".

In [10]:
%%dql -t show 
FROM df
SELECT *

┌─────────┬─────────┐
│ column1 │ column2 │
│  int64  │ varchar │
├─────────┼─────────┤
│       1 │ a       │
│       2 │ b       │
│       3 │ c       │
│       4 │ d       │
│       5 │ e       │
└─────────┴─────────┘



We can also change the default behavior by passing the "cell magic + type" without a query.

Let's set the extension default to DuckDB's default using the type "show".

In [11]:
%dql -t show

In [12]:
%%dql
FROM df
SELECT *

┌─────────┬─────────┐
│ column1 │ column2 │
│  int64  │ varchar │
├─────────┼─────────┤
│       1 │ a       │
│       2 │ b       │
│       3 │ c       │
│       4 │ d       │
│       5 │ e       │
└─────────┴─────────┘



### Doing 'Big Data' analysis. Bigger than RAM!
- DuckDB can also connect directly to files like CSV/Parquet

- We have a __huge__ file with emmissions: let's start working with it
- For starters, let's take advantage of DuckDB's "lazyness" and count the rows while avoiding loading all data into machine's memory

In [13]:
!pwd

/Users/migumax/Documents/My_projects/Udemy/duckdb/cases/4.duckdb_python_case1


In [13]:
%%dql
SELECT COUNT(*) 
FROM 'hourly_emissions_epacems.parquet'

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│    897900912 │
└──────────────┘



We could instantly count all those 897 million rows. It's like querying a database.

Let's now do 3 cool things: 
- get a number of descriptive statistics using avg & max filtering data from just one state
- do the same thing using Pandas
-compare execution time

In [14]:
%%time
%%dql
SELECT avg(co2_mass_tons) as average_co2_emmission, max(co2_mass_tons) as max_co2_emmission
FROM 'hourly_emissions_epacems.parquet'
WHERE state = 'CO'

┌───────────────────────┬───────────────────┐
│ average_co2_emmission │ max_co2_emmission │
│        double         │       float       │
├───────────────────────┼───────────────────┤
│     163.3414263103803 │            1047.0 │
└───────────────────────┴───────────────────┘

CPU times: user 363 ms, sys: 21 ms, total: 383 ms
Wall time: 144 ms


In [15]:
%%time
df = pd.read_parquet('hourly_emissions_epacems.parquet', engine='pyarrow', columns=['state', 'co2_mass_tons'])
print(df[df['state']=='CO']['co2_mass_tons'].mean(), df[df['state']=='CO']['co2_mass_tons'].max())

163.34154 1047.0
CPU times: user 10.8 s, sys: 5.69 s, total: 16.5 s
Wall time: 12.1 s


we can also display descriptive stats as a polars DataFrame

In [16]:
%%time
%%dql -t pl
SUMMARIZE 
SELECT * 
FROM 'hourly_emissions_epacems.parquet'
WHERE state = 'CO'

CPU times: user 14.7 s, sys: 511 ms, total: 15.2 s
Wall time: 3.55 s


column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
str,str,str,str,str,str,str,str,str,str,i64,str
"""plant_id_eia""","""INTEGER""","""465""","""56998""","""28""","""24658.32472912…","""25346.15655705…","""514""","""7068""","""55383""",13631472,"""0.0%"""
"""plant_id_epa""","""INTEGER""","""465""","""56998""","""28""","""24658.32472912…","""25346.15655705…","""514""","""7068""","""55383""",13631472,"""0.0%"""
"""emissions_unit…","""VARCHAR""","""1""","""S005""","""46""",null,null,null,null,null,13631472,"""0.0%"""
"""operating_date…","""TIMESTAMP WITH…","""1997-01-01 08:…","""2023-01-01 07:…","""223988""",null,null,null,null,null,13631472,"""0.0%"""
"""year""","""INTEGER""","""1997""","""2022""","""26""","""2011.085133579…","""6.882159196243…","""2006""","""2011""","""2017""",13631472,"""0.0%"""
"""state""","""VARCHAR""","""CO""","""CO""","""1""",null,null,null,null,null,13631472,"""0.0%"""
"""operating_time…","""FLOAT""","""0.0""","""1.0""","""87""","""0.480804509913…","""0.497078261513…","""0.0""","""0.028526293769…","""1.0""",13631472,"""0.64%"""
"""gross_load_mw""","""FLOAT""","""0.0""","""872.0""","""858""","""176.3127930569…","""145.8933029163…","""47.40620709755…","""139.8958239993…","""263.9244419968…",13631472,"""51.84%"""
"""heat_content_m…","""FLOAT""","""0.0""","""99999.9""","""172279""","""1722.750898236…","""1469.610654624…","""546.0258684088…","""1296.553674891…","""2482.671305558…",13631472,"""51.45%"""


Now let's group some data so we can create a visualization.

To store the query output into a variable, we can use "-o" followed by the variable name. Let's save it as a Polars DataFrame to the variable "df_gb".

In [17]:
%%time
%%dql -t pl -o df_gb
SELECT 
    year, 
    hour(operating_datetime_utc) AS hour, 
    SUM(co2_mass_tons) AS co2, 
FROM 'hourly_emissions_epacems.parquet'
WHERE state = 'CO'
GROUP BY ALL

CPU times: user 3.2 s, sys: 114 ms, total: 3.31 s
Wall time: 713 ms


year,hour,co2
i32,i64,f64
1998,23,1.6670e6
1998,4,1.6827e6
2006,12,1.7824e6
2006,20,2.0245e6
2006,3,2.0348e6
2006,6,1.9517e6
2013,23,1.9156e6
2013,4,1.9145e6
2009,21,1.8847e6
